In [1]:
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [2]:
! kaggle competitions download fake-news


 90% 42.0M/46.5M [00:01<00:00, 39.8MB/s]
100% 46.5M/46.5M [00:01<00:00, 28.1MB/s]


In [3]:
! unzip fake-news.zip

Archive:  fake-news.zip
  inflating: submit.csv              
  inflating: test.csv                
  inflating: train.csv               


In [4]:
import pandas as pd
import os


In [5]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

In [6]:
train.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [7]:
train.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [8]:
test.isnull().sum()

id          0
title     122
author    503
text        7
dtype: int64

In [9]:
train.shape

(20800, 5)

In [10]:
train = train.dropna() # dropping all the null values row

In [11]:
train.reset_index(inplace=True) # resetting index

In [45]:
trainf = train[['title']] # taking the feature we require to test

In [13]:
trainf.shape

(18285, 1)

In [46]:
trainf.reset_index(inplace=True)

In [47]:
y=train[['label']] # target variable

In [48]:
import nltk
import re
from nltk.corpus import stopwords

In [49]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

#Text Preprocessing
Here we are using stemming as it is easy and faster then we are removing all the stopwords replaicng all the spaces

In [50]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(trainf)):
    review = re.sub('[^a-zA-Z]', ' ', trainf['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

KeyboardInterrupt: 

In [ ]:
corpus


['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

#one hot encoding
Here we are doing one hot encoding with the voc size of 2000

In [19]:
from tensorflow.keras.preprocessing.text import one_hot
voc_size=2000
onehot_repr=[one_hot(words,voc_size)for words in corpus]

#Padding
As all the sentences may not be in the same length so we add a padding to be sure that none of the words miss.

In [20]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
sent_length=50
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ...  799 1158 1664]
 [   0    0    0 ... 2375 3295 3294]
 [   0    0    0 ... 2062  617 2119]
 ...
 [   0    0    0 ... 4391 1005   60]
 [   0    0    0 ...  613 3125 3964]
 [   0    0    0 ...  435 2024  608]]


In [21]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [23]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [24]:
model = RandomForestClassifier(max_depth=20,n_estimators=200)

# Train the model
model.fit(X_train, y_train)

<ipython-input-24-03f2b924fc2e>:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train, y_train)


RandomForestClassifier(max_depth=20, n_estimators=200)

In [25]:
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8937862468931235


In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[2830,  589],
       [  71, 2545]])

In [51]:
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8936205468102734


In [53]:
import pickle
filename = 'random_forest.sav'
pickle.dump(model, open(filename, 'wb'))

# load the model


In [26]:
test1= test[['text']]

In [31]:
type(trainf['title'][0])

str

In [27]:
test1.shape

(5200, 1)

In [28]:
len(test)

5200

In [30]:
from nltk.stem.porter import PorterStemmer
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
def text(t):
  if(len(t)==0):
    return 0
  ps = PorterStemmer()
  corpus = []


  review = re.sub('[^a-zA-Z]', ' ',t)
  review = review.lower()
  review = review.split()

  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)
  voc_size=5000
  onehot_repr=[one_hot(words,voc_size)for words in corpus]
  sent_length=20
  embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
  x=np.array(embedded_docs)
  return x

In [33]:
import csv

In [42]:
if type(test1['title'][9])==float:
  print("hello")

hello


In [44]:
with open('predictions.csv', 'w', newline='') as csvfile:
    fieldnames = ['id', 'label']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    for i in range(0, len(test)):
        t = test1['text'][i]
        if(type(t)==float):
          writer.writerow({'id': test['id'][i], 'label': 0})
        else:
          pro =  text(t)

          prediction = model.predict(pro)
          writer.writerow({'id': test['id'][i], 'label': prediction[0]})
